Лабораторная работа 3.

Обучить и сравнить качество нескольких предсказательных моделей в задаче регрессии. В лабораторной работе рассматривается задача предсказания выручки с каждого клиента магазина Gstore. Данные и подробное описание задачи может быть найдено тут https://www.kaggle.com/c/ga-customer-revenue-prediction/data (файл train.csv). При необходимости для ускорения процесса обучения можно взять только 10% случайных (!) строк данных. 

Задачи данной лабораторной работы:
- подобрать признаки (не менее 10, в т.ч. не менее 5 расчётных, отсутсвующих в данных явно), которые могут помочь в построении модели. По каждому расчитаному признаку пояснить, почему этот признак может быть важен. 
- обучить несколько классических моделей машинного обучения для задачи регрессии, включая GBM на дереьвях решений, лес решений, SVM, линейная регрессия. Зависимая переменная – логарифм от выручки с клиента 
- настроить гиперпараметры моделей. 
- с помощью 5-fold кросс валидации посчитать среднее качество каждой из моделей. 
- с помощью 20 случайных разбиений оценить дисперсию оценки каждой из моделей. 
- визуализировать полученные распределения метрик качества каждой из модели с помощью boxplotов, также отразить посчитанное ранее среднее качество.

In [ ]:
import numpy as np
import pandas as pd
import json

In [ ]:
#Считаем файл:
data = pd.read_csv("../input/ga-customer-revenue-prediction/train.csv", converters={'device': json.loads, 'geoNetwork': json.loads, 'totals': json.loads, 'trafficSource': json.loads}).sample(frac=0.2)

#Считаем столбцы в json'e:
device = pd.DataFrame(data['device'].tolist()).filter(items=['browser', 'isMobile', 'operatingSystem', 'deviceCategory'])
geoNetwork = pd.DataFrame(data['geoNetwork'].tolist()).filter(items=['continent', 'subContinent', 'country', 'region', 'metro', 'city', 'networkDomain'])
totals = pd.DataFrame(data['totals'].tolist())
trafficSource = pd.DataFrame(data['trafficSource'].tolist()).filter(items=['campaign', 'source', 'medium', 'keyword', 'isTrueDirect', 'referralPath', 'adContent'])

#Удаляем ненужные столбцы:
data = data.drop(labels = ['device', 'geoNetwork', 'totals', 'trafficSource', 'sessionId', 'visitId', 'fullVisitorId'], axis='columns')

#Склеим всё в один фрейм:
data = data.join(device).join(geoNetwork).join(totals).join(trafficSource)

In [ ]:
#Подготовка данных:
int_cols = ['visits', 'hits', 'pageviews', 'bounces', 'newVisits', 'transactionRevenue']
bool_cols = ['isTrueDirect']
string_cols = ['channelGrouping', 'deviceCategory', 'region', 'metro', 'city']
for col in int_cols:
    data[col].fillna(0, inplace=True)
for col in bool_cols:
    data[col].fillna(False, inplace=True)
for col in string_cols:
    data[col].fillna('(not set)')
    data.replace({ col: { 'not available in demo dataset': '(not set)'}})
    
data['socialEngagementType'] = data['socialEngagementType'] == 'Engaged'

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score

y = data['transactionRevenue']
x = data.filter(items = ['visits', 'hits', 'pageviews', 'bounces', 'newVisits'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [ ]:
models = [GradientBoostingRegressor(), RandomForestRegressor(), LinearRegression()]

In [ ]:
Result = pd.DataFrame()
tmp = pd.DataFrame()

#Кросс-валидация:
for model in models:
    model_name = str(model)
    scores = np.array([])
    for i in range(10):
        fold = KFold(n_splits=5, shuffle=True)
        score = cross_val_score(estimator=model, X=x, y=y, cv=fold, scoring='r2')
        scores = np.append(scores, score)
    tmp['Оценки качества'] = pd.Series(scores)
    tmp['Модель'] = model_name
    Result = Result.append(tmp)

In [ ]:
Result